In [ ]:
#radial velocities plotting
#importing
import numpy as np
import astropy.io.fits as fits
import matplotlib.pyplot as plt 
import pandas as pd
from astropy.stats import sigma_clip
from astropy.modeling import models,fitting
from scipy.ndimage import uniform_filter1d
from scipy.stats import norm
from scipy.special import voigt_profile
from scipy import signal
from scipy.signal import correlate
from PyAstronomy import pyasl
from scipy.optimize import curve_fit
from astropy.time import Time
import astropy.units as u
from astropy.coordinates import SkyCoord, EarthLocation
from astropy.constants import c
import astropy.io.fits as fits
from astropy.units import dimensionless_unscaled
from lmfit.models import LinearModel, GaussianModel, VoigtModel, PolynomialModel
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter
import glob
import fnmatch
import os
from astropy.timeseries import LombScargle
from plotnine import *
import traceback
import sys
import linecache
import textwrap
import re
import emcee
import corner
import tqdm

""" 
plt.rcParams['axes.linewidth'] = 1.25
plt.rcParams['xtick.major.size'] = 5
plt.rcParams['xtick.major.width'] = 1.25
plt.rcParams['ytick.major.size'] = 5
plt.rcParams['ytick.major.width'] = 1.25
 """
default_settings = {
    'font.size': 16,
    'axes.linewidth': 0.8,
    'xtick.major.size': 3.5,
    'xtick.major.width': 1,
    'ytick.major.size': 3.5,
    'ytick.major.width': 1
}


initial_settings = {
    'font.size': 22,
    'axes.linewidth': 1.25,
    'xtick.major.size': 5,
    'xtick.major.width': 1.25,
    'ytick.major.size': 5,
    'ytick.major.width': 1.25
}
plt.rcParams.update(initial_settings)

In [ ]:
#rv variations with time plotting
file_name_model = "/data/wdplanetary/omri/Output/resultfiles/SelfCCF/blue_spec_abs_lines.txt"
file_name_self = "/data/wdplanetary/omri/Output/resultfiles/bayesian/first_try_two_lines.txt"
times = []
delta_rvs = []
uerrors = []
lerrors = []
frequencies = []
powers = []

data_new = []
#normal textfiles:

data = np.loadtxt(file_name_self, delimiter = " ")
num_rows = data.shape[0]

sorted_indices = np.argsort(data[:, 0])
# Use sorted indices to sort the array
data = data[sorted_indices]

print(data.shape)
#data = np.delete((data), [0, 39],axis = 0)
print()

t = data[:,0]  # First column
mean = np.mean(data[:,1])
stdev = np.std(data[:,1])
v = data[:,1] - mean  # Second column
lerrs = np.abs(data[:,1] - data[:,2])   # Third column
uerrs = np.abs(data[:,3] - data[:,1])
#errs[errs > 5] = 5
for i in range(0,44):
    print(t[i],v[i],lerrs[i],uerrs[i])

print(mean,stdev)

variance = np.var(v) #+ np.mean(errs)**2
stdev = np.sqrt(variance)
#frequencies = np.linspace(1.95,2.05,3000)
frequencies = np.linspace(0.001,5,30000)
powers = LombScargle(t, v).power(frequencies)

 
fig, (ax_t, ax_w) = plt.subplots(2, 1, facecolor="white", figsize=(10,12), constrained_layout=True, dpi = 400)

ax_t.errorbar(t, v, yerr=(lerrs,uerrs),fmt = '.k',capsize=5,markersize=10,lw = 1)
ax_t.fill_between(t, -stdev, stdev, color='gray',label = "1 sigma", alpha=0.4)
ax_t.fill_between(t, -3* stdev, 3* stdev, color='gray',label = "3 sigma", alpha=0.2)
ax_t.axhline(y=0, color='black', linestyle='--', linewidth=1)
ax_t.legend()
#ax_t.text(np.max(t)/3, 2.5* stdev, f"RV mean = {mean:.3g} km/s")
ax_t.text(np.max(t)/3, 2.3* stdev, f'sigma = {stdev:.3g} km/s')
ax_t.set_xlabel("T - Days")
ax_t.set_ylabel("ΔRV - km/s")
""" 
#insrerting a planet
periodtime = np.linspace(np.min(t), np.max(t), 10000) 
ax_t.plot(periodtime, np.abs(v).max() * np.sin(2*np.pi* 2*periodtime))
ax_t.set_xlim(10,50)
 """
#plt.title("WD1929+012 Radial Velocity variations using Voigt fits - SALT data")
#plt.xticks(rotation=45)
#plt.tight_layout()
#plt.ylim(-20,20)
#plt.xlim(0,112)

normalized_powers = powers/(np.max(np.abs(powers)))
print(powers)
ax_w.plot(frequencies, normalized_powers )
ax_w.set_xlabel('Angular frequency [Periods/days]')
ax_w.set_ylabel('Normalized Power')
ax_w.tick_params(axis='x')
ax_w.tick_params(axis='y')

# Identify significant peaks (you can set your own threshold here)
threshold = 0.9 # Adjust as needed
significant_peak_indices = np.where(normalized_powers >= threshold)[0]
print(significant_peak_indices)
print(frequencies[1])
significant_peaks = np.array(frequencies[significant_peak_indices])

# Estimate false alarm rate for each peak
false_alarm_rates = []
for peak_freq in significant_peaks:
    # Use your preferred method to estimate false alarm rate here
    # Example: Monte Carlo simulations
    num_simulations = 100  # Adjust as needed
    peak_heights_simulated = []
    for _ in range(num_simulations):
        simulated_data = np.random.normal(0, 1, len(t))  # Generate random noise
        simulated_power = LombScargle(t, simulated_data).power([peak_freq])
        peak_heights_simulated.append(simulated_power[0])
    false_alarm_rate = np.sum(peak_heights_simulated >= normalized_powers[np.where(frequencies == peak_freq)]) / num_simulations
    false_alarm_rates.append(false_alarm_rate)

# Print significant peaks and their corresponding false alarm rates
print("Significant Peaks:")
for i, freq in enumerate(significant_peaks):
    print(f"Frequency: {freq}, False Alarm Rate: {false_alarm_rates[i]}")
 
os.makedirs("/data/wdplanetary/omri/Output/DeltaRV_files/SALT/bayesian/", exist_ok=True)
plt.savefig("/data/wdplanetary/omri/Output/DeltaRV_files/SALT/bayesian/firsttry.pdf")
plt.show()